In [ ]:
from random import randint
import numpy as np
def rand_bits(N):
  return [randint(0, 1) for i in range(N)]

In [ ]:
# Parameters
N = 64  # number of subcarriers
L = 4*N*3  # maybe try with non multiples to see how to make those work
max_delay = 10

In [ ]:
bit_stream = rand_bits(L)

In [ ]:
QAM_map = {
    (0,0,0,0):  1 + 1j,
    (0,0,0,1):  1 + 3j,
    (0,0,1,1):  1 - 3j,
    (0,0,1,0):  1 - 1j,

    (0,1,0,0):  3 + 1j,
    (0,1,0,1):  3 + 3j,
    (0,1,1,1):  3 - 3j,
    (0,1,1,0):  3 - 1j,

    (1,1,0,0): -3 + 1j,
    (1,1,0,1): -3 + 3j,
    (1,1,1,1): -3 - 3j,
    (1,1,1,0): -3 - 1j,

    (1,0,0,0): -1 + 1j,
    (1,0,0,1): -1 + 3j,
    (1,0,1,1): -1 - 3j,
    (1,0,1,0): -1 - 1j,
}

inverse_QAM_map = {v: k for k, v in QAM_map.items()}

In [ ]:
# turns list of bits into list of qam symbols
def QAM_mapping(bit_stream):
  L = len(bit_stream)
  bit_grps = [tuple(bit_stream[i:i+4]) for i in range(0, L, 4)]
  qam_symbols = [QAM_map[grp] for grp in bit_grps]
  return qam_symbols

def square_distance(cmplx1, cmplx2):
  return (cmplx1.real - cmplx2.real)**2 + (cmplx1.imag - cmplx2.imag)**2

# turns complex number into bit pattern corrsesponding to closest QAM symbol
def ML_Decoder(qam_symbol):
  QAM_Symbols = list(inverse_QAM_map.keys())
  min_index = np.argmin([square_distance(qam_symbol, sym) for sym in QAM_symbols])
  closest_symbol = QAM_symbols[min_index]
  return inverse_QAM_map[closest_symbol]

# Adds pad_length elements from end of qam_symbols to beginnging
def cyclic_prefix(qam_symbols, pad_length):
  return qam_symbols[-pad_length:] + qam_symbols

# Maps list of complex numbers to a list of bits
def inverse_QAM_mapping(qam_symbols):
  chunked = [ML_Decoder(sym) for sym in qam_symbols]
  return [bit for chunk in chunked for bit in chunk]

In [ ]:
QAM_symbols = QAM_mapping(bit_stream)
# Todo: add pilot carriers
chunked_QAM_symbols = [QAM_symbols[i:i+N] for i in range(0, len(QAM_symbols), N)]
time_domain_signal = [list(np.fft.ifft(chunk)) for chunk in chunked_QAM_symbols]

time_domain_signal_with_prefix = [cyclic_prefix(chunk, max_delay) for chunk in time_domain_signal]

In [ ]:
flattened_signal = [value for chunk in time_domain_signal_with_prefix for value in chunk]

In [ ]:
# pretend data has gone through a channel here
...
# Receiver
chunked_signal= [flattened_signal[i:i+N+max_delay] for i in range(0, len(flattened_signal), N+max_delay)]
prefix_free_signal = [chunk[max_delay:] for chunk in chunked_signal]
reconstructed_signal = [np.fft.fft(chunk) for chunk in prefix_free_signal]

reconstructed_symbols = [inverse_QAM_mapping(chunk) for chunk in reconstructed_signal]
reconstructed_bits = [bit for chunk in reconstructed_symbols for bit in chunk]

reconstructed_bits == bit_stream

True